In [1]:
!pip install .

Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple
Processing /root/autodl-tmp/DeepFRI-CAFA
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
ERROR: Could not install packages due to an OSError: [('/root/autodl-tmp/DeepFRI-CAFA/submission.tsv', '/tmp/pip-req-build-0l6z2p8y/submission.tsv', "[Errno 28] No space left on device: '/root/autodl-tmp/DeepFRI-CAFA/submission.tsv' -> '/tmp/pip-req-build-0l6z2p8y/submission.tsv'"), ('/root/autodl-tmp/DeepFRI-CAFA/testsuperset.fasta', '/tmp/pip-req-build-0l6z2p8y/testsuperset.fasta', "[Errno 122] Disk quota exceeded: '/tmp/pip-req-build-0l6z2p8y/testsuperset.fasta'"), ('/root/autodl-tmp

In [2]:
import os
os.chdir('./preprocessing')

DATA_DIR = './data/'
TFR_DIR = './data/ContactMaps/TFRecords'
SEQ_SIM = 95

os.makedirs(DATA_DIR, exist_ok=True)
print(f"\n\n DATA DIRECTORY ({DATA_DIR}) CREATED!\n")




 DATA DIRECTORY (./data/) CREATED!



In [ ]:
print("\n\n DOWNLOADING SIFTS-GO DATA...\n")
!wget ftp://ftp.ebi.ac.uk/pub/databases/msd/sifts/flatfiles/tsv/pdb_chain_go.tsv.gz -O ./data/pdb_chain_go.tsv.gz

print("\n\n DOWNLOADING SIFTS-EC DATA...\n")
!wget ftp://ftp.ebi.ac.uk/pub/databases/msd/sifts/flatfiles/tsv/pdb_chain_enzyme.tsv.gz -O ./data/pdb_chain_enzyme.tsv.gz

print("\n\n DOWNLOADING PDB SEQRES SEQUENCES...\n")
!wget ftp://ftp.wwpdb.org/pub/pdb/derived_data/pdb_seqres.txt.gz -O ./data/pdb_seqres.txt.gz

# print("\n\n DOWNLOADING PDB CLUSTERS...\n")
# !wget https://cdn.rcsb.org/resources/sequence/clusters/clusters-by-entity-95.txt -O ./data/bc-95.out

# print("\n\n DOWNLOADING GO HIERARCHY...\n")
!wget http://purl.obolibrary.org/obo/go/go-basic.obo -O ./data/go-basic.obo


In [ ]:
python PDB2distMap.py -annot ./data/CAFA__annot.tsv -seqres ./data/pdb_seqres.txt.gz -num_threads 30 -bc ./data/bc-95.out -out_dir ./data/annot_pdb_chains_npz/

In [2]:
print("\n\n  PREPROCESSING GO-ANNOTATIONS [Please wait this process may take a few minutes]...\n")
!python create_nrPDB_GO_annot.py -sifts ./data/pdb_chain_go.tsv.gz -bc ./data/bc-95.out -seqres ./data/pdb_seqres.txt.gz -obo ./data/go-basic.obo -out ./data/CAFA_



  PREPROCESSING GO-ANNOTATIONS [Please wait this process may take a few minutes]...



In [ ]:
!mkdir ./data/ContactMaps/
!mkdir ./data/ContactMaps/TFRecords/
!python PDB2TFRecord.py -annot ./data/CAFA__annot.tsv -prot_list ./data/CAFA__train.txt -npz_dir ./data/annot_pdb_chains_npz/ -num_shards 30 -num_threads 30 -tfr_prefix ./data/ContactMaps/TFRecords/PDB_GO_train
!python PDB2TFRecord.py -annot ./data/CAFA__annot.tsv -prot_list ./data/CAFA__valid.txt -npz_dir ./data/annot_pdb_chains_npz/ -num_shards 3 -num_threads 3 -tfr_prefix ./data/ContactMaps/TFRecords/PDB_GO_valid

In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

2023-06-01 21:35:34.317691: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-06-01 21:35:35.290039: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-06-01 21:35:35.326969: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-01 21:35:35.327495: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:06.0 name: Tesla V100S-PCIE-32GB computeCapability: 7.0
coreClock: 1.597GHz coreCount: 80 deviceMemorySize: 31.74GiB deviceMemoryBandwidth: 1.03TiB/s
2023-06-01 21:35:35.327523: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-06-01 21:35:35.346156: I tensorflow/stream_executor/plat

In [1]:
!python train_DeepCNN.py -ont bp -bs 64 --model_name ./trained_models/DeepCNN_BP --annot_fn ./preprocessing/data/CAFA2_annot.tsv --train_tfrecord_fn ./preprocessing/data/ContactMaps/TFRecords/PDB_GO_train --valid_tfrecord_fn ./preprocessing/data/ContactMaps/TFRecords/PDB_GO_valid

2023-06-18 21:43:29.721539: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-06-18 21:43:31.376753: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-06-18 21:43:31.488385: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-18 21:43:31.490313: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:07.0 name: NVIDIA A100-PCIE-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.45GiB deviceMemoryBandwidth: 1.41TiB/s
2023-06-18 21:43:31.490363: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-06-18 21:43:31.494197: I tensorflow/stream_executor/plat

In [2]:
!python train_DeepFRI.py -gc MultiGraphConv -bs 1 -ont bp --model_name ./trained_models/test --annot_fn ./preprocessing/data/CAFA2_annot.tsv -tf ./preprocessing/data/ContactMaps/TFRecords/PDB_GO_train -vf ./preprocessing/data/ContactMaps/TFRecords/PDB_GO_valid

Namespace(annot_fn='./preprocessing/data/CAFA2_annot.tsv', batch_size=1, cmap_thresh=10.0, cmap_type='ca', dropout=0.3, epochs=200, fc_dims=[2048], gc_dims=[512, 512, 512], gc_layer='MultiGraphConv', l2_reg=2e-05, lm_model_name='./trained_models/lstm_lm_tf.hdf5', lr=0.0002, model_name='./trained_models/test', ontology='bp', pad_len=None, test_list='./preprocessing/data/nrPDB-GO_2019.06.18_test.csv', train_tfrecord_fn='./preprocessing/data/ContactMaps/TFRecords/PDB_GO_train', valid_tfrecord_fn='./preprocessing/data/ContactMaps/TFRecords/PDB_GO_valid')
### Training model:  ./trained_models/test  on  3941  GO terms.
### Compiling DeepFRI model with MultiGraphConv layer...
Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
seq (InputLayer)                [(None, None, 26)]   0                                            
___

2023-07-02 09:43:59.475020: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'cudart64_101.dll'; dlerror: cudart64_101.dll not found
2023-07-02 09:43:59.475244: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-07-02 09:44:02.659049: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library nvcuda.dll
2023-07-02 09:44:02.679522: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3060 Laptop GPU computeCapability: 8.6
coreClock: 1.425GHz coreCount: 30 deviceMemorySize: 6.00GiB deviceMemoryBandwidth: 312.97GiB/s
2023-07-02 09:44:02.684632: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'cudart64_101.dll'; dlerror: cudart64_101.dll not found
2023-07-02 09:44:02.689848: W tensorflow/stre

In [ ]:
!python train_DeepCNN.py -ont cc -bs 64 --model_name ./trained_models/DeepCNN_CC --annot_fn ./preprocessing/data/CAFA2_annot.tsv --train_tfrecord_fn ./preprocessing/data/ContactMaps/TFRecords/PDB_GO_train --valid_tfrecord_fn ./preprocessing/data/ContactMaps/TFRecords/PDB_GO_valid

2023-06-18 23:21:57.851143: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-06-18 23:21:59.597603: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-06-18 23:21:59.744172: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-18 23:21:59.746085: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:07.0 name: NVIDIA A100-PCIE-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.45GiB deviceMemoryBandwidth: 1.41TiB/s
2023-06-18 23:21:59.746147: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-06-18 23:21:59.752261: I tensorflow/stream_executor/plat

In [ ]:
!python train_DeepFRI.py -gc MultiGraphConv -ont cc --model_name ./trained_models/MultiGCN_CC --annot_fn ./preprocessing/data/CAFA2_annot.tsv -tf ./preprocessing/data/ContactMaps/TFRecords/PDB_GO_train -vf ./preprocessing/data/ContactMaps/TFRecords/PDB_GO_valid

In [ ]:
!python train_DeepCNN.py -ont mf -bs 64 --model_name ./trained_models/DeepCNN_MF --annot_fn ./preprocessing/data/CAFA2_annot.tsv --train_tfrecord_fn ./preprocessing/data/ContactMaps/TFRecords/PDB_GO_train --valid_tfrecord_fn ./preprocessing/data/ContactMaps/TFRecords/PDB_GO_valid

In [ ]:
!python train_DeepFRI.py -gc MultiGraphConv -ont mf --model_name ./trained_models/MultiGCN_MF --annot_fn ./preprocessing/data/CAFA2_annot.tsv -tf ./preprocessing/data/ContactMaps/TFRecords/PDB_GO_train -vf ./preprocessing/data/ContactMaps/TFRecords/PDB_GO_valid

In [7]:
# p20536 test
!python predict.py --seq MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIPDKFFIQLKQPLRNKRVCVCGIDPYPKDGTGVPFESPNFTKKSIKEIASSISRLTGVIDYKGYNLNIIDGVIPWNYYLSCKLGETKSHAIYWDKISKLLLQHITKHVSVLYCLGKTDFSNIRAKLESPVTTIVGYHPAARDRQFEKDRSFEIINVLLELDNKVPINWAQGFIY -ont bp --verbose

2023-06-18 23:18:41.168104: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-06-18 23:18:42.602110: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-06-18 23:18:42.747756: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-18 23:18:42.749665: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:07.0 name: NVIDIA A100-PCIE-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.45GiB deviceMemoryBandwidth: 1.41TiB/s
2023-06-18 23:18:42.749711: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-06-18 23:18:42.755850: I tensorflow/stream_executor/plat

In [6]:
# Q9CQV8
!python predict.py --seq MTMDKSELVQKAKLAEQAERYDDMAAAMKAVTEQGHELSNEERNLLSVAYKNVVGARRSSWRVISSIEQKTERNEKKQQMGKEYREKIEAELQDICNDVLELLDKYLILNATQAESKVFYLKMKGDYFRYLSEVASGENKQTTVSNSQQAYQEAFEISKKEMQPTHPIRLGLALNFSVFYYEILNSPEKACSLAKTAFDEAIAELDTLNEESYKDSTLIMQLLRDNLTLWTSENQGDEGDAGEGEN -ont bp --verbose

2023-06-18 23:13:23.111848: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-06-18 23:13:24.592289: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-06-18 23:13:24.738367: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-18 23:13:24.740932: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:07.0 name: NVIDIA A100-PCIE-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.45GiB deviceMemoryBandwidth: 1.41TiB/s
2023-06-18 23:13:24.740997: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-06-18 23:13:24.757997: I tensorflow/stream_executor/plat

In [7]:
# A0A021WW32
!python predict.py --seq MFYEHIILAKKGPLARIWLAAHWDKKITKAHVFETNIEKSVEGILQPKVKLALRTSGHLLLGVVRIYSRKAKYLLADCNEAFVKIKMAFRPGMVDLPEGHREANVNAITLPEVFHDFDTALPELNDIDIEAQFSINQSRADEITMREDYGSLSLSLQDDGFGDIGFEAETPEIIRCSIPSNINDKIFDNDVLENIESLDPHSLDAHADMPGSRLDGDGFGDSFGQPALFEDDLFGDPSQPVEQITKESTTVLNADDSDEDAIDNIHNVPSPATSLVNSIEDEKEENNLNGHASVSENVPMNEITLVQNEDEGFALAPLDVSMYKGVTKAKRKRKLIIDEIKNISGEEMKAQLADTSDILTTLDLAPPTKRLMYWKETGGVEKLFSLPSRSIPARALFGNYNRQLFSHSTFFEDFSSVVPMEILALEFYTKENENALIIFNKKGRKRKNDNMSNLFLDHVPDSVVQSLEAPEVLRANHKSLGVSTVSVEIVSKEQESISCQNELTFFDNMRSPDLLSLNEMEQFSSINELPLTPRNMNHEMGDDFNQGDSTPAGLDHGHATPQHGNIGEMDHDSVIPTKKTAVILNESVGTSVLSDNGVSKRTNNILKGWDNYEIPSFVGQGIRHAGHCYQQYH -ont cc --verbose

2023-06-16 11:51:52.969245: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-06-16 11:51:54.678248: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-06-16 11:51:54.870608: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-16 11:51:54.872499: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:0c.0 name: NVIDIA A100-PCIE-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.45GiB deviceMemoryBandwidth: 1.41TiB/s
2023-06-16 11:51:54.872554: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-06-16 11:51:54.879970: I tensorflow/stream_executor/plat

In [ ]:
!python predict.py --fasta_fn testsuperset.fasta -ont mf

2023-06-16 18:03:53.710232: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-06-16 18:04:00.459152: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-06-16 18:04:00.636507: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-16 18:04:00.639733: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:0c.0 name: NVIDIA A100-PCIE-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.45GiB deviceMemoryBandwidth: 1.41TiB/s
2023-06-16 18:04:00.639862: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-06-16 18:04:00.788615: I tensorflow/stream_executor/plat

In [2]:
!pip install pandas
!pip install numpy==1.20.3

Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple
  Using cached https://repo.huaweicloud.com/repository/pypi/packages/9c/ee/77768cade9607687fadbcc1dcbb82dba0554154b3aa641f9c17233ffabe8/numpy-1.24.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-gpu 2.5.0 requires numpy~=1.19.2, but you have numpy 1.24.2 which is incompatible.
deepfri 0.0.1 requires numpy==1.18.5, but you have numpy 1.24.2 which is incompatible.
Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple
  Using cached https://repo.huaweicloud.com/repository/pypi/packages/a3/9d/5cc11fb882e628d21f8d490f56610f2e6612c9f55ecb25b2b57d4729c5

In [4]:
import numpy as np
import pandas as pd 
# linear = pd.read_csv('submission.tsv',
#     sep='\t')
# print('linear read')


In [ ]:
MF = pd.read_csv('DeepFRI_BP_predictions.csv', skiprows=1).iloc[:,[0,1,2]]
BP = pd.read_csv('DeepFRI_CC_predictions.csv', skiprows=1).iloc[:,[0,1,2]]
CC = pd.read_csv('DeepFRI_MF_predictions.csv', skiprows=1).iloc[:,[0,1,2]]
deepfri = pd.concat([MF,BP,CC])
del MF,BP,CC
deepfri.columns=['Id','GO term','Confidence']
print('deepfri read')
deepfri.head(5)

In [3]:
deepfri.to_csv('submission.tsv', sep='\t', index=False)

In [4]:
deepfri = deepfri.set_index(['Id', 'GO term'])
linear = linear.set_index(['Id', 'GO term'])

In [ ]:
# data = pd.merge(linear,deepfri,left_index=True, right_index=True, how='outer', suffixes=('', '_y'))
# del linear,deepfri
# gc.collect()
# print('merged')

In [ ]:
data['Confidence'] = data['Confidence'].fillna(data['Confidence_y'])
data = data.reset_index()
data = data.iloc[:,[0,1,2]]
print('data prepared')
data.shape

In [ ]:
data.to_csv('submission.tsv', sep='\t', index=False)
print('created submission')